# Borders data

### Постановка задачи

Для алгоритмического предсказания подходящей упаковки, будем использовать имеющиеся исторические данные о том, какие товары и в каком количества в какую коробку упаковывали. После, создадим удобные словари с данными об упаковках 

Структура словарей:

Ключи: трехбуквенные названия упаковок 

Значения: словари с данными
- borders – последовательность пар минимум-максимум для трех размерностей (ширина, длина, высота)
- vol_borders – объем товара, вычисленный как кубический объем из размерностей
- price – дополнительные данные о стоимости упаковки, используются для ранжирования упаковок и предложения самой дешевой

### Чтение данных

In [1]:
import pandas as pd
import json

Соединим данные из разных источников вместе – история заказов, стоимость упаковок и данные размеров упаковок

In [2]:
data = pd.read_csv('data_single_fixed.csv', index_col=0)

carton_price = pd.read_excel('carton_price.xlsx', index_col=0)
carton_price.columns = ['carton_price']

carton = pd.read_csv('carton.csv').drop('DISPLAYRFPACK', axis=1)

clear_data = data[~data['selected_cartontype'].isin(['STRETCH', 'NONPACK'])]


clear_data['selected_cartontype'].unique()

clear_data = pd.merge(data, 
                      carton_price, 
                      left_on='selected_cartontype', 
                      right_index=True)

Получим чистые данные для обработки

In [3]:
clear_data.head()

,selected_cartontype,goods_wght,sku,pack_class,a,b,c,0,40,120,...,900,910,950,960,970,sku_volume,min_sku_dim,max_sku_dim,density,carton_price
orderkey,,,,,,,,,,,,,,,,,,,,,
01277a58f3aa98c06db9046699ec162d,MYE,0.896,6287a74b664fd8ad4ed5c541b399a82b,nonpack,18.6,3.3,49.7,0,0,0,...,0,0,0,0,0,3051.0,3.3,49.7,0.000294,8.062722
2d809e93b73d1b19f9fa1e13d7fb62b9,MYE,2.078,eb0e661f3ed57ef7ca16418bf50e363e,nonpack,23.4,8.0,33.9,0,0,0,...,0,0,0,0,0,6346.0,8.0,33.9,0.000327,8.062722
f576f059bbcff2f6010a82176def2e3c,MYE,6.020,0588e43ea7ad1ef603059269db6467eb,nonpack,30.0,47.0,15.0,0,0,0,...,0,0,0,0,0,21150.0,15.0,47.0,0.000285,8.062722
12ec56c79b7da76818cc988881a6a2cd,MYE,6.080,706513014b99aa52bd13b2a59a97d354,nonpack,29.0,9.0,29.0,0,0,0,...,0,0,0,0,0,7569.0,9.0,29.0,0.000803,8.062722
aaae09e772f36a2277c0decb5ac99a43,MYE,1.087,22ebe748e358c7edde42bad4a7ce430a,nonpack,32.4,16.4,17.3,1,0,0,...,0,0,0,0,0,9193.0,16.4,32.4,0.000118,8.062722


### Словарь для одного товара

Воспользуемся имеющимися данными об упаковках, чтобы указать границы размеров товаров

In [4]:
# новые индексы для новых датафреймов
new_indexes = list(map('_'.join, (list(clear_data[['selected_cartontype', 
                                                   'a', 'b', 'c', 'sku_volume']]
                                       .groupby('selected_cartontype')
                                       .agg(['min', 'max'])
                                       .columns))))

# группируем по типу упаковки, выбирая наименьшую и наибольшую размерность
grouped_clear_data = (clear_data[['selected_cartontype', 
                                  'a', 'b', 'c', 'sku_volume']]
                      .groupby('selected_cartontype')
                      .agg(['min', 'max'])
                      .droplevel(0, 1))

grouped_clear_data.columns = new_indexes

grouped_clear_data = (pd.merge(grouped_clear_data, carton_price, 
                               left_index=True, right_index=True))

По итогу получаем сводную таблицу

In [5]:
grouped_clear_data

,a_min,a_max,b_min,b_max,c_min,c_max,sku_volume_min,sku_volume_max,carton_price
MYA,0.1,70.0,0.01,86.0,0.1,94.0,1.0,133920.0,1.109861
MYB,0.1,100.0,0.01,84.0,0.1,95.0,2.0,268128.0,2.297432
MYC,0.2,150.0,0.01,97.0,0.1,82.0,2.0,105000.0,3.616713
MYD,0.2,150.0,0.10,86.0,0.5,85.0,18.0,110500.0,6.918375
MYE,0.3,160.0,0.01,97.0,0.8,94.0,14.0,96000.0,8.062722
MYF,1.0,88.0,0.50,34.9,1.0,65.0,28.0,108680.0,4.083130
YMA,1.0,116.0,0.30,66.0,0.4,47.7,12.0,96000.0,4.392937
YMC,1.3,150.0,1.00,57.5,1.0,77.0,28.0,136080.0,7.777487
YME,1.0,150.0,3.00,38.0,4.0,60.0,102.0,46483.0,23.670260
YMF,5.0,54.0,0.30,40.0,5.0,42.7,234.0,25044.0,10.661487


Собираем словарь, в котором будут храниться данные о границах

In [6]:
# в словаре хранится инфа о свойствах упаковок
result_dct = {}

for i in grouped_clear_data.index:
    result_dct[i] = {}
    # оч хитрая схема, я сортирую минимакс размеры, чтобы сравнивать с размерами товара один раз 
    result_dct[i]['borders'] = tuple(sorted(((grouped_clear_data.loc[i].a_min, grouped_clear_data.loc[i].a_max), 
                                             (grouped_clear_data.loc[i].b_min, grouped_clear_data.loc[i].b_max), 
                                             (grouped_clear_data.loc[i].c_min, grouped_clear_data.loc[i].c_max)), 
                                            key=lambda x: x[1]))
    result_dct[i]['vol_borders'] = tuple(sorted((grouped_clear_data.loc[i].sku_volume_min, grouped_clear_data.loc[i].sku_volume_max)))
    result_dct[i]['price'] = grouped_clear_data.loc[i].carton_price
    
result_dct = {i: result_dct[i] for i in sorted(result_dct, key=lambda x: result_dct[x]['price'])}

json.dump(result_dct, open('types_dict.json', 'w'))

Пример элемента словаря

In [7]:
result_dct['MYA']

{'borders': ((0.1, 70.0), (0.01, 86.0), (0.1, 94.0)),
 'vol_borders': (1.0, 133920.0),
 'price': 1.10986120605469}

### Словарь для двух товаров

Воспользуемся историческими данными, вычисляя размерности коробок, в которые когда либо клали два товара, чтобы узнать границы

In [9]:
# чтение данных
data = pd.read_csv('data.csv', index_col=0)

# заказы с несколькими товарами(2-3)
orders_few_items = data.groupby('orderkey')['sku'].count()\
                .reset_index().query('sku!=1')['orderkey'].to_list()

# заказы где box_num имеет только одно уникальное значение
orders_one_box = data.groupby('orderkey')['box_num'].nunique()\
                .reset_index().query('box_num==1')['orderkey'].to_list()

#заказы где несколько товаров но все в одной коробке
data_few_goods = data[(data.orderkey.isin(orders_few_items))&(data.orderkey.isin(orders_one_box))]

#есть несколько заказов(всего 736 строк) где box_num не 1, отбросим их, скорее всего какая-то ошибка выгрузки 
data_few_goods = data_few_goods[data_few_goods.box_num == 1]

#добавим размеры товаров
sku = pd.read_csv('sku.csv', index_col=0)

#используем inner join так как не все товары в data есть в таблице sku
data_few_goods = data_few_goods.merge(sku, on='sku', how='inner')

#добавим новую колонку sku_count с количеством товаров в заказе
data_few_goods = data_few_goods.merge(
    data_few_goods.groupby('orderkey')['sku']\
    .count().reset_index().rename({'sku':'sku_count'}, axis=1),
    on='orderkey', how='left')

# заказы только с двумя товарами
data_two_goods = data_few_goods[data_few_goods.sku_count==2].copy()
data_two_goods = data_two_goods.drop(['whs','box_num', 'recommended_cartontype',
                                      'selected_carton', 'sel_calc_cube','recommended_carton',
                                     'pack_volume', 'rec_calc_cube', 'who', 'trackingid', 'sku_count'], axis=1)
data_two_goods['sku_vol'] = data_two_goods['a']*data_two_goods['b']*data_two_goods['c']

two_goods_borders = (data_two_goods[['selected_cartontype', 'a', 'b', 'c', 'sku_vol']]
                     .groupby('selected_cartontype')
                     .agg(['min', 'max'])
                     .droplevel(0, axis=1))
two_goods_borders.columns = ['a_min', 'a_max', 'b_min', 'b_max', 'c_min', 'c_max', 'vol_min', 'vol_max']
two_goods_borders = (pd.merge(carton_price, two_goods_borders, 
                              left_index=True, right_index=True)
                     .sort_values(by='carton_price'))

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


По итогу получаем сводную таблицу

In [10]:
two_goods_borders

,carton_price,a_min,a_max,b_min,b_max,c_min,c_max,vol_min,vol_max
MYA,1.109861,0.30,58.0,0.10,37.0,0.1,40.0,1.50000,44770.000
MYB,2.297432,0.00,70.0,0.00,56.0,0.0,47.7,0.00000,96000.000
MYC,3.616713,0.30,70.0,0.10,44.0,0.2,47.7,5.33000,47250.000
MYF,4.083130,1.30,22.0,0.30,25.0,1.0,20.2,14.00000,6387.240
YMA,4.392937,0.00,50.0,0.00,34.8,0.0,40.0,0.00000,36000.000
MYD,6.918375,0.00,60.0,0.00,44.0,0.0,60.0,0.00000,63070.000
YMC,7.777487,0.93,60.0,0.10,40.0,0.3,60.0,10.00000,60000.000
MYE,8.062722,0.00,67.0,0.00,59.0,0.0,60.6,0.00000,37206.000
YMF,10.661487,1.00,59.0,0.50,43.0,0.2,49.0,36.00000,19175.000
YMW,13.870000,0.90,58.0,0.01,44.0,2.0,47.0,16.00000,31250.000


Создаем словарь, в котором данные будут храниться для передачи json

In [11]:
dct = {}
two_goods_borders_dict = two_goods_borders.to_dict(orient='index')
for i in two_goods_borders_dict:
    dct[i] = {'borders': [], 'vol_borders': []}
    dct[i]['borders'].extend([[two_goods_borders_dict[i]['a_min'], two_goods_borders_dict[i]['a_max']], 
                              [two_goods_borders_dict[i]['b_min'], two_goods_borders_dict[i]['b_max']],
                              [two_goods_borders_dict[i]['c_min'], two_goods_borders_dict[i]['c_max']]])
    dct[i]['vol_borders'] = [round(two_goods_borders_dict[i]['vol_min'], 2), 
                                  round(two_goods_borders_dict[i]['vol_max'], 2)]
    dct[i]['price'] = two_goods_borders_dict[i]['carton_price']

Пример элемента словаря

In [12]:
dct['MYA']

{'borders': [[0.3, 58.0], [0.1, 37.0], [0.1, 40.0]],
 'vol_borders': [1.5, 44770.0],
 'price': 1.10986120605469}

### Словарь для трех товаров

По такому же принципу сделаем словари для 3 и 4+ товаров

In [13]:
# заказы только с тремя товарами
data_three_goods = data_few_goods[data_few_goods.sku_count==3].copy()

data_three_goods = data_three_goods.drop(['whs','box_num', 'recommended_cartontype',
                                      'selected_carton', 'sel_calc_cube','recommended_carton',
                                     'pack_volume', 'rec_calc_cube', 'who', 'trackingid', 'sku_count'], axis=1)

data_three_goods['sku_vol'] = data_three_goods['a']*data_three_goods['b']*data_three_goods['c']

three_goods_borders = (data_three_goods[['selected_cartontype', 'a', 'b', 'c', 'sku_vol']]
                     .groupby('selected_cartontype')
                     .agg(['min', 'max'])
                     .droplevel(0, axis=1))
three_goods_borders.columns = ['a_min', 'a_max', 'b_min', 'b_max', 'c_min', 'c_max', 'vol_min', 'vol_max']
three_goods_borders = (pd.merge(carton_price, three_goods_borders, 
                              left_index=True, right_index=True)
                     .sort_values(by='carton_price'))

dct_3 = {}
two_goods_borders_dict = two_goods_borders.to_dict(orient='index')
for i in two_goods_borders_dict:
    dct_3[i] = {'borders': [], 'vol_borders': []}
    dct_3[i]['borders'].extend([[two_goods_borders_dict[i]['a_min'], two_goods_borders_dict[i]['a_max']], 
                              [two_goods_borders_dict[i]['b_min'], two_goods_borders_dict[i]['b_max']],
                              [two_goods_borders_dict[i]['c_min'], two_goods_borders_dict[i]['c_max']]])
    dct_3[i]['vol_borders'] = [round(two_goods_borders_dict[i]['vol_min'], 2), 
                                  round(two_goods_borders_dict[i]['vol_max'], 2)]
    dct_3[i]['price'] = two_goods_borders_dict[i]['carton_price']

In [14]:
dct_3['MYA']

{'borders': [[0.3, 58.0], [0.1, 37.0], [0.1, 40.0]],
 'vol_borders': [1.5, 44770.0],
 'price': 1.10986120605469}

### Словарь для 4+ товаров

In [15]:
# заказы только с четырмя товарами
data_four_goods = data_few_goods[data_few_goods.sku_count>=4].copy()
data_four_goods = data_four_goods.drop(['whs','box_num', 'recommended_cartontype',
                                      'selected_carton', 'sel_calc_cube','recommended_carton',
                                     'pack_volume', 'rec_calc_cube', 'who', 'trackingid', 'sku_count'], axis=1)

data_four_goods['sku_vol'] = data_four_goods['a']*data_four_goods['b']*data_four_goods['c']

four_goods_borders = (data_four_goods[['selected_cartontype', 'a', 'b', 'c', 'sku_vol']]
                     .groupby('selected_cartontype')
                     .agg(['min', 'max'])
                     .droplevel(0, axis=1))
four_goods_borders.columns = ['a_min', 'a_max', 'b_min', 'b_max', 'c_min', 'c_max', 'vol_min', 'vol_max']
four_goods_borders = (pd.merge(carton_price, four_goods_borders, 
                              left_index=True, right_index=True)
                     .sort_values(by='carton_price'))

dct_4 = {}
four_goods_borders_dict = four_goods_borders.to_dict(orient='index')
for i in four_goods_borders_dict:
    dct_4[i] = {'borders': [], 'vol_borders': []}
    dct_4[i]['borders'].extend([[four_goods_borders_dict[i]['a_min'], four_goods_borders_dict[i]['a_max']], 
                              [four_goods_borders_dict[i]['b_min'], four_goods_borders_dict[i]['b_max']],
                              [four_goods_borders_dict[i]['c_min'], four_goods_borders_dict[i]['c_max']]])
    dct_4[i]['vol_borders'] = [round(four_goods_borders_dict[i]['vol_min'], 2), 
                                  round(four_goods_borders_dict[i]['vol_max'], 2)]
    dct_4[i]['price'] = four_goods_borders_dict[i]['carton_price']

In [16]:
dct_4['MYA']

{'borders': [[0.1, 61.5], [0.1, 37.0], [0.1, 44.8]],
 'vol_borders': [3.25, 24192.0],
 'price': 1.10986120605469}

### Сбор всех данных вместе

Для удобства, соберем все полученные словари в один большой, положив их в порядке индексов

In [18]:
result = {'data': [None, # отбойник для 0 товаров
                   # далее следуют по порядку (индексов) данные для 1-4+ товаров
                   result_dct, 
                   dct, 
                   dct_3, 
                   dct_4]}

Пример: словарь для двух товаров на индексе 2

In [19]:
result['data'][2]

{'MYA': {'borders': [[0.3, 58.0], [0.1, 37.0], [0.1, 40.0]],
  'vol_borders': [1.5, 44770.0],
  'price': 1.10986120605469},
 'MYB': {'borders': [[0.0, 70.0], [0.0, 56.0], [0.0, 47.7]],
  'vol_borders': [0.0, 96000.0],
  'price': 2.29743245442708},
 'MYC': {'borders': [[0.3, 70.0], [0.1, 44.0], [0.2, 47.7]],
  'vol_borders': [5.33, 47250.0],
  'price': 3.61671256510417},
 'MYF': {'borders': [[1.3, 22.0], [0.3, 25.0], [1.0, 20.2]],
  'vol_borders': [14.0, 6387.24],
  'price': 4.08313023663949},
 'YMA': {'borders': [[0.0, 50.0], [0.0, 34.8], [0.0, 40.0]],
  'vol_borders': [0.0, 36000.0],
  'price': 4.39293721516927},
 'MYD': {'borders': [[0.0, 60.0], [0.0, 44.0], [0.0, 60.0]],
  'vol_borders': [0.0, 63070.0],
  'price': 6.91837518601191},
 'YMC': {'borders': [[0.93, 60.0], [0.1, 40.0], [0.3, 60.0]],
  'vol_borders': [10.0, 60000.0],
  'price': 7.77748710436699},
 'MYE': {'borders': [[0.0, 67.0], [0.0, 59.0], [0.0, 60.6]],
  'vol_borders': [0.0, 37206.0],
  'price': 8.06272243923611},
 'YM

In [20]:
json.dump(result, open('borders_data.json', 'w'))